#Try Stock Prediction - NETFLIX

In [21]:
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import datetime as dt
import joblib
import yfinance as yf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

## 1. Read Data

In [22]:
# stock_data = pd.read_csv('/content/NFLX.csv', index_col='Date')
# stock_data

ticker = 'ETH-USD'
# start_date = '2020-05-20'
# end_date = pd.Timestamp.now().strftime('%Y-%m-%d')
start_date = '2020-05-21'
end_date = '2024-05-21'
stock_data = yf.download(ticker, start=start_date, end=end_date)
stock_data

[*********************100%%**********************]  1 of 1 completed


Open         High          Low        Close    Adj Close  \
Date                                                                          
2020-05-21   210.129150   211.625183   193.346436   199.883606   199.883606   
2020-05-22   199.837112   208.591537   198.040863   207.169189   207.169189   
2020-05-23   207.194489   210.386459   205.294220   208.694397   208.694397   
2020-05-24   208.716064   210.595078   202.370346   202.370346   202.370346   
2020-05-25   201.982651   206.361450   200.667557   205.319748   205.319748   
...                 ...          ...          ...          ...          ...   
2024-05-16  3036.014160  3041.807129  2925.087402  2945.131104  2945.131104   
2024-05-17  2945.136963  3120.301758  2934.112549  3094.118652  3094.118652   
2024-05-18  3094.553467  3146.790527  3087.704346  3122.948975  3122.948975   
2024-05-19  3122.824707  3137.148438  3056.754395  3071.843018  3071.843018   
2024-05-20  3071.858887  3690.805908  3050.297852  3663.855469  3663.855469   

                 Volume  
Date                     
2020-05-21  13308321229  
2020-05-22  12041592114  
2020-05-23  10665476768  
2020-05-24  11833299572  
2020-05-25  10415044124  
...                 ...  
2024-05-16  13035465176  
2024-05-17  14449438097  
2024-05-18   9407051320  
2024-05-19   8747800800  
2024-05-20  31228143948  

[1461 rows x 6 columns]

## 2. Plotting Data

### 2.1 Plot High & Low

In [23]:
plt.figure(figsize=(15,10))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=60))
# x_dates = [dt.datetime.strptime(d, '%Y-%m-%d').date() for d in stock_data.index.values]
x_dates = stock_data.index

plt.plot(x_dates, stock_data['High'], label='High')
plt.plot(x_dates, stock_data['Low'], label='Low')
plt.xlabel('Time Scale')
plt.ylabel('Scaled USD')
plt.legend()
plt.gcf().autofmt_xdate()
plt.show()

### 2.2 Plot Open & Close

In [24]:
plt.figure(figsize=(15,10))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=60))
# x_dates = [dt.datetime.strptime(d, '%Y-%m-%d').date() for d in stock_data.index.values]
x_dates = stock_data.index

plt.plot(x_dates, stock_data['Open'], label='Open')
plt.plot(x_dates, stock_data['Close'], label='Close')
plt.xlabel('Time Scale')
plt.ylabel('Scaled USD')
plt.legend()
plt.gcf().autofmt_xdate()
plt.show()

## 3. Stock Prediction

### 3.1 Normalize or Rescale

In [25]:
features = stock_data.iloc[:, 0:4]
sc = StandardScaler()
features_scaled = sc.fit_transform(features.values)
stock_data_ft = pd.DataFrame(columns=features.columns, data=features_scaled, index=features.index)

# Simpan objek StandardScaler
joblib.dump(sc, 'standard_scaler.pkl')

['standard_scaler.pkl']

### 3.2 Train & Test

#### 3.2.1 Splitting

In [26]:
def lstm_split(data, n_steps):
  X, y = [], []
  for i in range(len(data) - n_steps):
    X.append(data[i:i + n_steps, :-1])
    y.append(data[i + n_steps-1, -1])

  return np.array(X), np.array(y)

In [37]:
n_steps = 20
X1, y1 = lstm_split(stock_data_ft.values, n_steps=n_steps)

train_split = 0.8
split_idx = int(np.ceil(len(X1)*train_split))
date_index = stock_data_ft.index

X_train, X_test = X1[:split_idx], X1[split_idx:]
y_train, y_test = y1[:split_idx], y1[split_idx:]
X_train_date, X_test_date = date_index[n_steps-1:split_idx + n_steps-1], date_index[split_idx + n_steps-1:len(X1) + n_steps-1]

print(X1.shape, X_train.shape, X_test.shape, y_test.shape)

(1441, 20, 3) (1153, 20, 3) (288, 20, 3) (288,)


#### 3.2.2 Training

In [48]:
lstm = Sequential()
lstm.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu', return_sequences=True))
lstm.add(LSTM(50, activation="relu"))
lstm.add(Dense(1))
lstm.compile(loss='mean_squared_error', optimizer='adam')
lstm.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_22 (LSTM)              (None, 20, 50)            10800     
                                                                 
 lstm_23 (LSTM)              (None, 50)                20200     
                                                                 
 dense_11 (Dense)            (None, 1)                 51        
                                                                 
Total params: 31051 (121.29 KB)
Trainable params: 31051 (121.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [57]:
history = lstm.fit(X_train, y_train, epochs=150, batch_size=4, verbose=2, shuffle=False)

Epoch 1/150
289/289 - 8s - loss: 0.0101 - 8s/epoch - 27ms/step
Epoch 2/150
289/289 - 4s - loss: 0.0107 - 4s/epoch - 13ms/step
Epoch 3/150
289/289 - 5s - loss: 0.0111 - 5s/epoch - 18ms/step
Epoch 4/150
289/289 - 4s - loss: 0.0143 - 4s/epoch - 14ms/step
Epoch 5/150
289/289 - 4s - loss: 0.0136 - 4s/epoch - 14ms/step
Epoch 6/150
289/289 - 5s - loss: 0.0115 - 5s/epoch - 17ms/step
Epoch 7/150
289/289 - 4s - loss: 0.0131 - 4s/epoch - 15ms/step
Epoch 8/150
289/289 - 4s - loss: 0.0117 - 4s/epoch - 14ms/step
Epoch 9/150
289/289 - 4s - loss: 0.0111 - 4s/epoch - 15ms/step
Epoch 10/150
289/289 - 5s - loss: 0.0155 - 5s/epoch - 17ms/step
Epoch 11/150
289/289 - 4s - loss: 0.0134 - 4s/epoch - 13ms/step
Epoch 12/150
289/289 - 4s - loss: 0.0099 - 4s/epoch - 14ms/step
Epoch 13/150
289/289 - 5s - loss: 0.0095 - 5s/epoch - 18ms/step
Epoch 14/150
289/289 - 4s - loss: 0.0086 - 4s/epoch - 14ms/step
Epoch 15/150
289/289 - 4s - loss: 0.0087 - 4s/epoch - 14ms/step
Epoch 16/150
289/289 - 5s - loss: 0.0087 - 5s/epo

In [58]:
plt.plot(history.history['loss'])
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

#### 3.2.3 Evaluation

In [59]:
# Make predictions
y_pred = lstm.predict(X_test)

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(X_test_date, y_test, label='True Values', color='blue')
plt.plot(X_test_date, y_pred, label='Predicted Values', color='red')
plt.title('True Values vs Predicted Values')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

9/9 [==============================] - 0s 11ms/step


In [60]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("RMSE: ", rmse)
print("MAPE: ", mape)

RMSE:  0.046259267879896086
MAPE:  0.0802079498475205


In [61]:
# Save Model
lstm.save("predict_nflx.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#### 3.2.4 Compare Performance

In [34]:
n_steps_list = [2, 6, 13, 20]
results = []
models = []

for n_steps in n_steps_list:
    X1, y1 = lstm_split(stock_data_ft.values, n_steps=n_steps)

    split_idx = int(np.ceil(len(X1) * 0.8))
    X_train, X_test = X1[:split_idx], X1[split_idx:]
    y_train, y_test = y1[:split_idx], y1[split_idx:]

    lstm = Sequential()
    lstm.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu', return_sequences=True))
    lstm.add(LSTM(50, activation='relu'))
    lstm.add(Dense(1))
    lstm.compile(loss='mean_squared_error', optimizer='adam')

    lstm.fit(X_train, y_train, epochs=100, batch_size=4, verbose=2, shuffle=False)

    y_pred = lstm.predict(X_test)
    y_pred = y_pred.flatten()  # Ensure y_pred is a 1D array

    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    results.append((n_steps, rmse, mape))
    models.append(lstm)

Epoch 1/100
292/292 - 3s - loss: 0.3482 - 3s/epoch - 9ms/step
Epoch 2/100
292/292 - 1s - loss: 0.0255 - 797ms/epoch - 3ms/step
Epoch 3/100
292/292 - 1s - loss: 0.0131 - 788ms/epoch - 3ms/step
Epoch 4/100
292/292 - 1s - loss: 0.0147 - 772ms/epoch - 3ms/step
Epoch 5/100
292/292 - 1s - loss: 0.0113 - 945ms/epoch - 3ms/step
Epoch 6/100
292/292 - 1s - loss: 0.0103 - 1s/epoch - 4ms/step
Epoch 7/100
292/292 - 1s - loss: 0.0085 - 1s/epoch - 4ms/step
Epoch 8/100
292/292 - 1s - loss: 0.0073 - 1s/epoch - 4ms/step
Epoch 9/100
292/292 - 1s - loss: 0.0063 - 857ms/epoch - 3ms/step
Epoch 10/100
292/292 - 1s - loss: 0.0055 - 790ms/epoch - 3ms/step
Epoch 11/100
292/292 - 1s - loss: 0.0050 - 802ms/epoch - 3ms/step
Epoch 12/100
292/292 - 1s - loss: 0.0047 - 801ms/epoch - 3ms/step
Epoch 13/100
292/292 - 1s - loss: 0.0048 - 791ms/epoch - 3ms/step
Epoch 14/100
292/292 - 1s - loss: 0.0047 - 883ms/epoch - 3ms/step
Epoch 15/100
292/292 - 1s - loss: 0.0051 - 922ms/epoch - 3ms/step
Epoch 16/100
292/292 - 1s - los

In [35]:
for result in results:
    print(f"n_steps: {result[0]}, RMSE: {result[1]}, MAPE: {result[2]}")

n_steps: 2, RMSE: 0.10181539352104418, MAPE: 0.1084884086001867
n_steps: 6, RMSE: 0.05560561007003294, MAPE: 0.10546221053083095
n_steps: 13, RMSE: 0.04839442306291657, MAPE: 0.08426254370995145
n_steps: 20, RMSE: 0.03392554224755317, MAPE: 0.07224729834753443


In [36]:
plt.figure(figsize=(10, 5))
plt.plot([result[0] for result in results], [result[1] for result in results], marker='o', label='RMSE')
plt.plot([result[0] for result in results], [result[2] for result in results], marker='o', label='MAPE')
plt.xlabel('n_steps')
plt.ylabel('Error')
plt.title('Error vs. n_steps')
plt.legend()
plt.show()

The result shows that n_steps = n will reach the most accurate prediction, means that predicting the n+1th days is the best choice

#### 3.2.5 Cross Validation

In [ ]:
# n_steps = 6
# X1, y1 = lstm_split(stock_data_ft.values, n_steps=n_steps)

In [ ]:
# tscv = TimeSeriesSplit(n_splits=5)
# rmse_scores = []
# mape_scores = []

# for train_index, test_index in tscv.split(X1):
#     X_train, X_test = X1[train_index], X1[test_index]
#     y_train, y_test = y1[train_index], y1[test_index]

#     # Define the model
#     model = Sequential()
#     model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu', return_sequences=True))
#     model.add(LSTM(50, activation='relu'))
#     model.add(Dense(1))
#     model.compile(loss='mean_squared_error', optimizer='adam')

#     # Train the model
#     model.fit(X_train, y_train, epochs=100, batch_size=4, verbose=0, shuffle=False)

#     # Predict
#     y_pred = model.predict(X_test)
#     y_pred = y_pred[:, 0]

#     # Calculate RMSE and MAPE
#     rmse = mean_squared_error(y_test, y_pred, squared=False)
#     mape = mean_absolute_percentage_error(y_test, y_pred)

#     rmse_scores.append(rmse)
#     mape_scores.append(mape)

# print(f"Average RMSE: {np.mean(rmse_scores)}, Average MAPE: {np.mean(mape_scores)}")

###3.3 Run Saved Model

In [62]:
# Load the SavedModel
model = load_model('predict_nflx.h5')
sc = joblib.load('standard_scaler.pkl')

In [63]:
def prepare_input_data(ticker, end_date, n_steps):
    stock_data = yf.download(ticker, end=end_date)
    X_feat = stock_data[['Open', 'High', 'Low']]
    target_y = stock_data['Close']

    X_ft = sc.fit_transform(X_feat.values)
    stock_data_ft = pd.DataFrame(columns=X_feat.columns, data=X_ft, index=X_feat.index)
    stock_data_ft['Close'] = target_y.values

    input_data = stock_data_ft.values[-n_steps:, :-1]
    input_data = np.expand_dims(input_data, axis=0)

    return input_data, stock_data_ft.index[-1]

In [67]:
# Parameter prediksi
ticker = 'ETH-USD'
end_date = '2024-05-01'
n_steps = 20

# Menyiapkan data input
input_data, last_date = prepare_input_data(ticker, end_date, n_steps)

# Melakukan prediksi
predicted_price_scaled = model.predict(input_data).flatten()[0]

# Mengembalikan hasil prediksi ke skala semula
predicted_price = sc.inverse_transform([[np.nan, predicted_price_scaled, np.nan]])[0][1]

# Menampilkan hasil prediksi
print(f"Harga saham {ticker} yang diprediksi untuk tanggal {end_date} adalah: {predicted_price:.2f}")

[*********************100%%**********************]  1 of 1 completed

1/1 [==============================] - 0s 23ms/step
Harga saham ETH-USD yang diprediksi untuk tanggal 2024-05-01 adalah: 2927.79
